<a href="https://colab.research.google.com/github/Danalmestadi/T5-Capstone/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*

#install packages 📦

In [ ]:
!pip install timm
!pip install ultralytics
!pip install transformers

#Importing libraries 📚

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO


#Loading the dataset 🧷

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="D4wlgnOPMyBMaKBdCkPA")
project = rf.workspace("t5-xhhs7").project("car-collision_system")
version = project.version(2)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.3, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Car-Collision_System-2 in yolov8:: 100%|██████████| 33352/33352 [00:09<00:00, 3639.49it/s]


# Training The YOLO Model 🔶

In [ ]:
!yolo task=detect mode=predict model='/content/final_model.pt' conf=0.50 source='/content/Dataset_Capstone.mp4' save=True

Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 186 layers, 2,685,148 parameters, 0 gradients, 6.8 GFLOPs

video 1/1 (frame 1/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 140.8ms
video 1/1 (frame 2/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 122.6ms
video 1/1 (frame 3/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 116.1ms
video 1/1 (frame 4/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 112.1ms
video 1/1 (frame 5/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 110.1ms
video 1/1 (frame 6/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 120.4ms
video 1/1 (frame 7/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 107.6ms
video 1/1 (frame 8/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 122.2ms
video 1/1 (frame 9/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 121.6ms
video 1/1 (frame 10/141) /content/Dataset_Capstone.mp4: 640x384 1 car, 112.5ms
video 1/1 (frame 11/141) /content/Dataset_Capstone.mp4: 640

--------


Manually label each image to find out the safe distance threshold.

In [ ]:
#Making the video path as global varible
video_capture = '/content/Video.mp4'

In [ ]:
# Function to initialize the video source
def initialize_video_source(source):
    global video_capture
    video_capture = cv2.VideoCapture(source)
    if not video_capture.isOpened():
        print("Error opening video source")
        return False
    return True

#Loading the MiDas Model 🔷

In [ ]:
# Load the MiDaS model for depth estimation
def load_midas_model():
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
    midas.to('cuda' if torch.cuda.is_available() else 'cpu')
    midas.eval()
    midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
    transform = midas_transforms.small_transform
    return midas, transform

#Capturing Frame 📸

In [ ]:

# Capture a frame from the video
def capture_frame():
    global video_capture
    if video_capture is None:
        print("Video source not initialized")
        return None
    ret, frame = video_capture.read()
    if ret:
        return frame
    else:
        print("Failed to capture frame")
        return None

#Function to utlize Trained Yolo Model 🔍

In [ ]:
# YOLO model for object detection
def model_object_detection_predict(frame, yolo_model):
    results = yolo_model(frame)
    return results

#Function to utlize Depth estimation model 🛠

In [ ]:
def model_depth_estimation_predict(frame, midas, transform, yolo_bboxes):
    # Convert the frame to RGB and prepare it for depth estimation
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Run the depth estimation model
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    # Convert the prediction to a normalized depth map
    depth_map = prediction.cpu().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)

    # Loop through YOLO bounding boxes and calculate distances
    distances = []
    for bbox in yolo_bboxes:
        x_min, y_min, x_max, y_max = map(int, bbox)  # Cast bbox coordinates to integers

        # Slice the depth map for the region inside the bounding box
        bbox_depth = depth_map[y_min:y_max, x_min:x_max]

        # Calculate the mean or median depth within the bounding box
        avg_depth = np.mean(bbox_depth)

        # Append the result for this bounding box
        distances.append(avg_depth)

    return depth_map, distances

#### Function to calculate the average depth for a bounding box 🔎


In [ ]:
# Function to calculate the average depth for a bounding box
def calculate_average_depth(depth_map, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    roi_depth = depth_map[y1:y2, x1:x2]
    return np.mean(roi_depth)

#Main Function to test models and Save the Video 🟥

In [ ]:
# Main function to process video and save output with depth estimation and object detection
def test_models_and_save_video():
    safe_distance = 0.2  # Adjust this value based on your depth map scale

    if not initialize_video_source('/content/Video2.mp4'):
        return

    midas, transform = load_midas_model()
    yolo_model = YOLO('/content/final_model.pt')  # Load YOLO model once

    # Get video properties
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Create VideoWriter object
    out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    frame_count = 0
    while True:
        frame = capture_frame()
        if frame is None:
            break

        frame_count += 1
        print(f"Processing frame {frame_count}")

        # Object detection
        od_results = model_object_detection_predict(frame, yolo_model)
        yolo_bboxes = [det[:4].tolist() for det in od_results[0].boxes.data]

        # Depth estimation
        depth_map, distances = model_depth_estimation_predict(frame, midas, transform, yolo_bboxes)

        # Visualize depth map
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_map, alpha=255), cv2.COLORMAP_JET)

        # Blend original frame with depth colormap
        blended = cv2.addWeighted(frame, 0.6, depth_colormap, 0.4, 0)

        # Process results
        for i, det in enumerate(od_results[0].boxes.data):
            bbox = det[:4].tolist()
            conf = det[4].item()
            cls = int(det[5].item())

            avg_depth = distances[i]

            # Draw bounding box
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(blended, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Add text
            label = f"Class: {cls}, Conf: {conf:.2f}, Depth: {avg_depth:.4f}"
            cv2.putText(blended, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            if avg_depth < safe_distance:
                warning = "WARNING: Too Close!"
                cv2.putText(blended, warning, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Write the frame
        out.write(blended)

        # Optional: break after processing a certain number of frames
        if frame_count >= 300:  # Process 10 seconds assuming 30 fps
            break

    video_capture.release()
    out.release()
    print("Processing complete. Output saved as 'output_video.mp4'")

# Run the test and save the video
test_models_and_save_video()

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Processing frame 1



Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


0: 384x640 (no detections), 201.6ms
Speed: 3.4ms preprocess, 201.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2

0: 384x640 (no detections), 193.3ms
Speed: 3.2ms preprocess, 193.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3

0: 384x640 (no detections), 181.4ms
Speed: 3.8ms preprocess, 181.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4

0: 384x640 (no detections), 190.8ms
Speed: 3.7ms preprocess, 190.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5

0: 384x640 (no detections), 188.5ms
Speed: 3.4ms preprocess, 188.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6

0: 384x640 (no detections), 202.0ms
Speed: 3.2ms preprocess, 202.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 7

0: 384x640 (no detections), 199.2ms
Speed: 3.3ms preprocess, 199.2ms inferen